In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime

pd.set_option('display.max_columns', 100)

import psycopg2

# 0. DataLoad

In [11]:
conn = psycopg2.connect("host=172.32.1.126 port=5432 user=linkshops password=$dpdlvmflf4 dbname=LINKSHOPS-20211121")

curs = conn.cursor()

In [12]:
from_date = '2021-05-01'
to_date = '2022-05-01'

## 1) Users

In [4]:
SQL = "SELECT \"id\"," + (
        "\"name\",") + (
        "\"roles\",") + (
        "\"createdAt\",") + ( 
        "\"updatedAt\",") + (
        "\"type\",") + (
        "\"isActive\",") + (
        "\"lastLoginAt\" FROM users")

curs.execute(SQL)

dataframe = pd.DataFrame(curs)

dataframe.columns = ['buyerId','name','roles','createdAt','updatedAt','type', 'isActive', 'lastLoginAt']

## 2) tb_update_history

In [5]:
SQL = "SELECT \"target_id\"," + (
        "\"key\",") + (
        "\"before\",") + (
        "\"after\",") + ( 
        "\"updated_at\" FROM tb_update_history ") + (
        "where \"key\"='membership_buyer'")

curs.execute(SQL)

dataframe_history = pd.DataFrame(curs)

if dataframe_history.shape[0] > 1:
    dataframe_history.columns = ['buyerId','key','before','after','historyUpdatedAt']
else :
    dataframe_history = DataFrame(columns = ['buyerId','key','before','after','historyUpdatedAt'])

## 3) wait_shipment_logs

In [7]:
'''
SQL = "select \"buyerId\"," + (
    "\"brandId\", ") + ( 
    "\"orderProductId\",") + ( 
    "\"quantity\",") + ( 
    "\"finalQuantity\",") + ( 
    "\"userQuantity\",") + ( 
    "\"pickupQuantity\",") + ( 
    "\"isLatest\"" ) + ( 
    " from wait_shipment_logs where ") + (
    "\"isLatest\" = 'Y' AND ") + (
     f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}'")

curs.execute(SQL)

dataframe_shipment = pd.DataFrame(curs)

dataframe_shipment.columns = ['buyerId',
                            'brandId',
                            'orderProductId',
                            'quantity',
                            'finalQuantity',
                            'userQuantity',
                            'pickupQuantity',
                            'isLatest']
'''

'\nSQL = "select "buyerId"," + (\n    ""brandId", ") + ( \n    ""orderProductId",") + ( \n    ""quantity",") + ( \n    ""finalQuantity",") + ( \n    ""userQuantity",") + ( \n    ""pickupQuantity",") + ( \n    ""isLatest"" ) + ( \n    " from wait_shipment_logs where ") + (\n    ""isLatest" = \'Y\' AND ") + (\n     f""createdAt" > \'{from_date}\' AND ") + (\n    f""createdAt" < \'{to_date}\'")\n\ncurs.execute(SQL)\n\ndataframe_shipment = pd.DataFrame(curs)\n\ndataframe_shipment.columns = [\'buyerId\',\n                            \'brandId\',\n                            \'orderProductId\',\n                            \'quantity\',\n                            \'finalQuantity\',\n                            \'userQuantity\',\n                            \'pickupQuantity\',\n                            \'isLatest\']\n'

## 4) order_products

In [8]:
'''
SQL = "select \"buyerId\"," + (
    "\"brand\", ") + ( 
    "\"totalKRW\",") + ( 
    "\"finalTotalKRW\",") + ( 
    "\"finalQuantity\",") + ( 
    "\"quantity\"") + ( 
    " from order_products WHERE ") + (
    f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}'")

SQL = "select * " + ( 
    " from order_products WHERE ") + (
    f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}'")

curs.execute(SQL)

dataframe_order = pd.DataFrame(curs)

dataframe_order.columns = ['buyerId',
                           'brand',
                           'cusumOrderAmount',
                           'cusumPurchaseAmount',
                           'cusumPurchaseQuantity',
                           'cusumOrderQuantity']
'''

'\nSQL = "select "buyerId"," + (\n    ""brand", ") + ( \n    ""totalKRW",") + ( \n    ""finalTotalKRW",") + ( \n    ""finalQuantity",") + ( \n    ""quantity"") + ( \n    " from order_products WHERE ") + (\n    f""createdAt" > \'{from_date}\' AND ") + (\n    f""createdAt" < \'{to_date}\'")\n\nSQL = "select * " + ( \n    " from order_products WHERE ") + (\n    f""createdAt" > \'{from_date}\' AND ") + (\n    f""createdAt" < \'{to_date}\'")\n\ncurs.execute(SQL)\n\ndataframe_order = pd.DataFrame(curs)\n\ndataframe_order.columns = [\'buyerId\',\n                           \'brand\',\n                           \'cusumOrderAmount\',\n                           \'cusumPurchaseAmount\',\n                           \'cusumPurchaseQuantity\',\n                           \'cusumOrderQuantity\']\n'

### 5) Order

In [13]:
SQL = "select * " + ( 
    " from orders WHERE ") + (
    f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}'")

curs.execute(SQL)

dataframe_order = pd.DataFrame(curs)

In [16]:
SQL = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'orders'"

curs.execute(SQL)

dataframe_order.columns = [i[0] for i in curs]

## 5) tb_order_fee

In [17]:
SQL = "select * from tb_order_fee WHERE " + (
    f"\"created_at\" > '{from_date}' AND ") + (
    f"\"created_at\" < '{to_date}'")

curs.execute(SQL)

dataframe_fee = DataFrame(curs)

SQL = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'tb_order_fee'"

curs.execute(SQL)

dataframe_fee.columns = [i[0] for i in curs]

dataframe_fee.iloc[:,9:] = dataframe_fee.iloc[:,9:].astype('float')

# 1. 데이터 EDA

## 1) 데이터 현황 파악

### (1) 결측값 기반 분석

In [20]:
dataframe_fee.apply(lambda x : x.isna().value_counts())

id  created_at  creator_id  updated_at  updater_id  use_yn  \
False  152993      152993      152993      152993      152993  152993   

       order_id  is_tax_free_user  product_amount  product_supply  \
False    152993            152993          152993          152993   

       product_vat  dc_product_amount  dc_product_supply  dc_product_vat  \
False       152993             152993             152993          152993   

       handling_fee_amount  handling_fee_supply  handling_fee_vat  \
False               152993               152993            152993   

       quality_inspection_fee_amount  quality_inspection_fee_supply  \
False                         152993                         152993   

       quality_inspection_fee_vat  stock_fee_amount  stock_fee_supply  \
False                      152993            152993            152993   

       stock_fee_vat  dc_handling_fee_amount  dc_handling_fee_supply  \
False         152993                  152993                  152993   

       dc_handling_fee_vat  dc_quality_inspection_fee_amount  \
False               152993                            152993   

       dc_quality_inspection_fee_supply  dc_quality_inspection_fee_vat  \
False                            152993                         152993   

       dc_stock_fee_amount  dc_stock_fee_supply  dc_stock_fee_vat  \
False               152993               152993            152993   

       order_shipping_cost_amount  order_shipping_cost_supply  \
False                      152993                      152993   

       order_shipping_cost_vat  wait_shipment_shipping_cost_amount  \
False                   152993                              152993   

       wait_shipment_shipping_cost_supply  wait_shipment_shipping_cost_vat  \
False                              152993                           152993   

       dc_order_shipping_cost_amount  dc_order_shipping_cost_supply  \
False                         152993                         152993   

       dc_order_shipping_cost_vat  dc_wait_shipment_shipping_cost_amount  \
False                      152993                                 152993   

       dc_wait_shipment_shipping_cost_supply  \
False                                 152993   

       dc_wait_shipment_shipping_cost_vat  additional_payment_amount  \
False                              152993                     152993   

       additional_payment_supply  additional_payment_vat  \
False                     152993                  152993   

       dc_additional_payment_amount  dc_additional_payment_supply  \
False                        152993                        152993   

       dc_additional_payment_vat  
False                     152993

결측값은 존재하지 않는다.

### (2) 분포 기반 확인

In [22]:
dataframe_fee.describe()

id     creator_id     updater_id      order_id  \
count  152993.000000  152993.000000  152993.000000  1.529930e+05   
mean   459446.000000  101467.275699  101467.275699  1.089416e+06   
std     44165.419204   38690.847093   38690.847093  4.863772e+04   
min    382950.000000    1217.000000    1217.000000  1.007866e+06   
25%    421198.000000   69999.000000   69999.000000  1.047827e+06   
50%    459446.000000  111401.000000  111401.000000  1.087487e+06   
75%    497694.000000  137056.000000  137056.000000  1.127399e+06   
max    535942.000000  148644.000000  148644.000000  1.177070e+06   

       product_supply   product_vat  dc_product_amount  dc_product_supply  \
count    1.529930e+05  1.529930e+05      152993.000000       152993.00000   
mean     1.783884e+05  1.081286e+04           0.679933            0.61812   
std      7.171516e+05  2.089791e+04         181.053308          164.59408   
min      2.000000e+00  0.000000e+00           0.000000            0.00000   
25%      3.600000e+04  2.100000e+03           0.000000            0.00000   
50%      7.800000e+04  5.700000e+03           0.000000            0.00000   
75%      1.620000e+05  1.280000e+04           0.000000            0.00000   
max      4.371400e+07  1.119600e+06       68685.000000        62441.00000   

       dc_product_vat  handling_fee_amount  handling_fee_supply  \
count   152993.000000         1.529930e+05         1.529930e+05   
mean         0.061813         8.795455e+03         8.453703e+03   
std         16.459228         5.255326e+04         5.249878e+04   
min          0.000000         0.000000e+00        -3.800000e+01   
25%          0.000000         1.240000e+03         1.128000e+03   
50%          0.000000         2.760000e+03         2.536000e+03   
75%          0.000000         6.100000e+03         5.620000e+03   
max       6244.000000         3.497120e+06         3.497120e+06   

       handling_fee_vat  quality_inspection_fee_amount  \
count     152993.000000                   1.529930e+05   
mean         341.752858                   9.480416e+03   
std          660.501901                   4.646631e+04   
min           -9.000000                   0.000000e+00   
25%           63.000000                   1.200000e+03   
50%          178.000000                   2.800000e+03   
75%          407.000000                   6.500000e+03   
max        36915.000000                   3.185000e+06   

       quality_inspection_fee_supply  quality_inspection_fee_vat  \
count                   1.529930e+05               152993.000000   
mean                    9.147442e+03                  332.973437   
std                     4.640300e+04                  680.371466   
min                    -4.500000e+03                 -141.000000   
25%                     1.091000e+03                   45.000000   
50%                     2.548000e+03                  173.000000   
75%                     6.001000e+03                  391.000000   
max                     3.185000e+06                36257.000000   

       stock_fee_amount  stock_fee_supply  stock_fee_vat  \
count          152993.0          152993.0       152993.0   
mean                0.0               0.0            0.0   
std                 0.0               0.0            0.0   
min                 0.0               0.0            0.0   
25%                 0.0               0.0            0.0   
50%                 0.0               0.0            0.0   
75%                 0.0               0.0            0.0   
max                 0.0               0.0            0.0   

       dc_handling_fee_amount  dc_handling_fee_supply  dc_handling_fee_vat  \
count            1.529930e+05            1.529930e+05        152993.000000   
mean             6.028967e+02            5.902947e+02            12.601923   
std              8.305940e+03            8.287868e+03           121.806417   
min              0.000000e+00            0.000000e+00             0.000000   
25%              0.000000e+00           

< 각 컬럼별 의미 정리 >

- product : 상품가 관련
- dc_product : 상품 할인 관련


- handling_fee : 판매수수료 관련
- dc_handling_fee : 판매수수료 할인 관련


- quality_inspection : 검품수수료 관련
- dc_quality_inspection : 검품수수료 할인 관련


- stock_fee : 상품 보관료 관련
- dc_stock_fee : 상품 보관료 할인 관련


- order_shipping_cost : 주문배송비 관련
- dc_order_shipping_cost : 주문배송비 할인 관련


- wait_shipment_shipping_cost : 미송배송비 관련
- dc_wait_shipment_shipping_cost 미송배송비 할인 관련


- additional_payment : 추가결제금액 관련
- dc_additional_payment : 추가결제금액 할인 관련

< 분포 인사이트 >

- dc_product 관련 값들은 거의 대부분의 값이 0이다.
- dc_handling 관련 값들은 거의 대부분의 값이 0이다.
- dc_quality_inspection  관련 값들은 거의 대부분의 값이 0이다.
- dc_order_shipping_cost 관련 값들은 거의 대부분이 0이다.
- dc_additional_payment 관련 값들은 거의 대부분이 0이다.


- wait_shipment_shipping_cost 관련 값들은 거의 대부분이 0이다.
- additional_payment 관련 값들은 대부분이 0이다.


- stock_fee 관련 값들은 전부 0이다.  
- dc_stock_fee 관련 값들은 전부 0이다.


결론을 내리자면, 

- 할인 관련된 값들은 거의 대부분의 값이 0인 분포이고, 
- 상품 보관료, 미송배송비는 거의 대부분이 0이다.

분포 기반 분석에서 도출한 추가적으로 탐색해볼만한 주제로는 

- 각 컬럼 주제(ex. 판매수수료, 검품수수료, 상품보관료 ...)별 속해있는 컬럼들의 관계성 파악
- 할인 관련 컬럼들과 값 관련 컬럼들의 관계성 파악
- 모든 값들을 포괄하는 새로운 파생변수 생성이 가능한지 여부 결정

### (3) Unique한 ID는 무엇인가

In [24]:
dataframe_fee.apply(lambda x : len(pd.unique(x)) / x.shape[0])

id                                       1.000000
created_at                               0.999935
creator_id                               0.078363
updated_at                               0.999935
updater_id                               0.078363
use_yn                                   0.000007
order_id                                 1.000000
is_tax_free_user                         0.000013
product_amount                           0.035969
product_supply                           0.027544
product_vat                              0.019981
dc_product_amount                        0.000033
dc_product_supply                        0.000033
dc_product_vat                           0.000033
handling_fee_amount                      0.044512
handling_fee_supply                      0.094429
handling_fee_vat                         0.021315
quality_inspection_fee_amount            0.020458
quality_inspection_fee_supply            0.042466
quality_inspection_fee_vat               0.019550


### 결론 : id와, order_id는 모든 행에 대하여 유니크한 값으로 보인다.

(가칭) order_fee_id와 order_id는 1:1 연동 관계로 보인다.

## 2) 각 컬럼 주제별 속해있는 컬럼들의 관계성 파악

#### (1) 가설 : product_amount는 product_supply와 product_vat을 더한 금액일것이다.

상품공급액은 상품 자체의 가격과, 거기에 붙은 VAT를 더한 값에서 차이가 없어야한다.

그렇지 않은 경우 이유 탐색이 필요하다.

In [19]:
(dataframe_fee['product_amount'] == (dataframe_fee['product_supply'] + dataframe_fee['product_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다. 

### (2) 가설 2 : handling_fee_amount는 가설1과 마찬가지로 supply와 vat을 더한 금액일것이다.

가설1과 마찬가지로, 상품공급액은 공급액과 vat를 더한 값에서 차이가 없어야한다.

In [23]:
(dataframe_fee['handling_fee_amount'] == (
    dataframe_fee['handling_fee_supply'] + dataframe_fee['handling_fee_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다.

### (3) 가설 3 : quality_inspection_fee는 가설1과 마찬가지로 supply와 vat을 더한 금액일것이다.

In [24]:
(dataframe_fee['quality_inspection_fee_amount'] == (
    dataframe_fee['quality_inspection_fee_supply'] + dataframe_fee['quality_inspection_fee_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다.

### (4) 가설 4 : order_shipping_cost는 가설1과 마찬가지로 supply와 vat을 더한 금액일것이다.

In [26]:
(dataframe_fee['order_shipping_cost_amount'] == (
    dataframe_fee['order_shipping_cost_supply'] + dataframe_fee['order_shipping_cost_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다.

### (5) 가설 5 : wait_shipment_shipping_cost는 가설1과 마찬가지로 supply와 vat을 더한 금액일것이다.

In [30]:
(dataframe_fee['wait_shipment_shipping_cost_amount'] == (
    dataframe_fee['wait_shipment_shipping_cost_supply'] + dataframe_fee['wait_shipment_shipping_cost_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다.

### (6) 가설 6 :additional_payment는 가설1과 마찬가지로 supply와 vat을 더한 금액일것이다.

In [32]:
(dataframe_fee['additional_payment_amount'] == (
    dataframe_fee['additional_payment_supply'] + dataframe_fee['additional_payment_vat'])).value_counts()

True    152993
dtype: int64

- 결론 : 가설은 정확히 일치한다.

## 3) 교차 분석

### (1) orders와 교차분석

#### - 가설을 세우지 않고, 밑바닥부터 자유롭게 탐색한다.

In [21]:
dataframe_order.columns

Index(['id', 'buyerId', 'address', 'shippingCost', 'status', 'totalPaid',
       'paymentStatus', 'processedDate', 'createdAt', 'updatedAt', 'version',
       'totalCNY', 'totalKRW', 'totalUSD', 'subtotalCNY', 'subtotalKRW',
       'subtotalUSD', 'taxCNY', 'taxKRW', 'taxUSD', 'handlingFeeCNY',
       'handlingFeeKRW', 'handlingFeeUSD', 'shippingCostCNY',
       'shippingCostKRW', 'shippingCostUSD', 'finalSubtotalCNY',
       'finalSubtotalKRW', 'finalSubtotalUSD', 'finalTaxCNY', 'finalTaxKRW',
       'finalTaxUSD', 'finalHandlingFeeCNY', 'finalHandlingFeeKRW',
       'finalHandlingFeeUSD', 'finalShippingCostCNY', 'finalShippingCostKRW',
       'finalShippingCostUSD', 'finalTotalCNY', 'finalTotalKRW',
       'finalTotalUSD', 'adjustmentTotalCNY', 'adjustmentTotalKRW',
       'adjustmentTotalUSD', 'finalAdjustmentTotalCNY',
       'finalAdjustmentTotalKRW', 'finalAdjustmentTotalUSD',
       'settlementStatus', 'orderedAt', 'skipNotification', 'orderStageStatus',
       'initHandlingFeeRa

In [22]:
dataframe_fee.columns

Index(['id', 'created_at', 'creator_id', 'updated_at', 'updater_id', 'use_yn',
       'order_id', 'is_tax_free_user', 'product_amount', 'product_supply',
       'product_vat', 'dc_product_amount', 'dc_product_supply',
       'dc_product_vat', 'handling_fee_amount', 'handling_fee_supply',
       'handling_fee_vat', 'quality_inspection_fee_amount',
       'quality_inspection_fee_supply', 'quality_inspection_fee_vat',
       'stock_fee_amount', 'stock_fee_supply', 'stock_fee_vat',
       'dc_handling_fee_amount', 'dc_handling_fee_supply',
       'dc_handling_fee_vat', 'dc_quality_inspection_fee_amount',
       'dc_quality_inspection_fee_supply', 'dc_quality_inspection_fee_vat',
       'dc_stock_fee_amount', 'dc_stock_fee_supply', 'dc_stock_fee_vat',
       'order_shipping_cost_amount', 'order_shipping_cost_supply',
       'order_shipping_cost_vat', 'wait_shipment_shipping_cost_amount',
       'wait_shipment_shipping_cost_supply', 'wait_shipment_shipping_cost_vat',
       'dc_order_shippin

위에서 살펴봤듯이, dataframe_fee는 7개 주제의 값(상품가, 판매수수료, 검품수수료, 상품 보관료, 주문배송비, 미송배송비, 추가결제금액) 으로 이루어져 있다.

각 주제별 값들과 dataframe_order간의 관계를 파악하면 모든 값을 포괄하는 파생변수 생성에 대한 인사이트를 얻을 수 있을지도 모른다.

In [37]:
dataframe_fee[dataframe_fee['dc_product_amount'] > 0]

id                       created_at  creator_id  \
114653  423191 2021-06-22 09:15:02.035155+00:00       87097   
114681  423189 2021-06-22 09:13:49.607868+00:00       87097   
114692  423186 2021-06-22 09:12:45.099119+00:00       87097   
114693  423187 2021-06-22 09:13:20.904836+00:00       87097   
114698  423199 2021-06-22 09:20:06.079123+00:00       87097   
114703  423207 2021-06-22 09:31:03.856455+00:00       87097   

                             updated_at  updater_id use_yn  order_id  \
114653 2021-06-22 09:15:02.035155+00:00       87097      Y   1049898   
114681 2021-06-22 09:13:49.607868+00:00       87097      Y   1049896   
114692 2021-06-22 09:12:45.099119+00:00       87097      Y   1049893   
114693 2021-06-22 09:13:20.904836+00:00       87097      Y   1049894   
114698 2021-06-22 09:20:06.079123+00:00       87097      Y   1049906   
114703 2021-06-22 09:31:03.856455+00:00       87097      Y   1049918   

        is_tax_free_user product_amount  product_supply  product_vat  \
114653             False      122760.00        111600.0      11160.0   
114681             False      122760.00        111600.0      11160.0   
114692             False      912515.00        829559.0      82956.0   
114693             False      122760.00        111600.0      11160.0   
114698             False      125700.00        114273.0      11427.0   
114703             False      130680.00        118800.0      11880.0   

        dc_product_amount  dc_product_supply  dc_product_vat  \
114653             9240.0             8400.0           840.0   
114681             9240.0             8400.0           840.0   
114692            68685.0            62441.0          6244.0   
114693             9240.0             8400.0           840.0   
114698             6300.0             5727.0           573.0   
114703             1320.0             1200.0           120.0   

        handling_fee_amount  handling_fee_supply  handling_fee_vat  \
114653               4360.0               3963.0             397.0   
114681               4360.0               3963.0             397.0   
114692              32390.0              29443.0            2947.0   
114693               4360.0               3963.0             397.0   
114698               4360.0               3963.0             397.0   
114703               2180.0               1981.0             199.0   

        quality_inspection_fee_amount  quality_inspection_fee_supply  \
114653                         4900.0                         4457.0   
114681                         4900.0                         4457.0   
114692                        33600.0                        30563.0   
114693                         4900.0                         4457.0   
114698                         4900.0                         4457.0   
114703                         4900.0                         4457.0   

        quality_inspection_fee_vat  stock_fee_amount  stock_fee_supply  \
114653                       443.0               0.0               0.0   
114681                       443.0               0.0               0.0   
114692                      3037.0               0.0               0.0   
114693                       443.0               0.0               0.0   
114698                       443.0               0.0               0.0   
114703                       443.0               0.0               0.0   

        stock_fee_vat  dc_handling_fee_amount  dc_handling_fee_supply  \
114653            0.0                     0.0                     0.0   
114681            0.0                     0.0                     0.0   
114692            0.0                     0.0                     0.0   
114693            0.0                     0.0                     0.0   
114698            0.0                     0.0                     0.0   
114703            0.0                  2180.0                  1982.0   

        dc_handling_fee_vat  dc_quality_inspection_fee_amount  \
114653             

In [ ]:
dataframe_fee.loc[dataframe_fee['dc_product_amount'] > 0]

'mao'